# SiRF Reader - Locosys SiRF Binary

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
from math import log10, floor

import unittest

from file_reader import FileReader

## Main Class

In [2]:
class SirfReader(FileReader):
    '''SBN + SBP files - Locosys SiRF Binary'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)
        
        self.device = None


    def load(self, headerOnly=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            if headerOnly == False:
                self.buffer = f.read()
            else:
                self.buffer = f.read(64)

            if len(self.buffer) > 0:
                self.readHeader()
                if headerOnly == False:
                    self.readData()


    def readHeader(self):
        '''Read Locosys header into memory'''

        # Main header content - ignore the checksum because people have been known to patch the username
        message = self.readMessage(evaluateChecksum=False)

        # Parse ASCII data, ignoring message[0] which is the SiRF message ID - typically 0xfd but sometimes 0xdf
        splitFields = message[1:].decode('ascii').split(',')
        
        # Decode fields that are always present, regardless of the message ID being 0xfd or 0xdf
        self.header['username'] = splitFields[0]
        try:
            self.header['serial'] = int(splitFields[1])
        except:
            # This handles the rare case of an SBP file with the string "unknown,unknown,1,unknown"
            self.header['serial'] = 0
        self.header['frequency'] = 5 if splitFields[2] == '2' else 1
        self.header['firmware'] = splitFields[3]

        # Note that message ID 0xdf has 2 additional fields (maybe 3) which are currently ignored
        # e.g. "GW60USER,168605168,2,V1.3A0926C,0.0h,00,"

        self.device = getDevice(self.header['serial'])


    def readMessage(self, evaluateChecksum=True):
        '''Read SiRF message from buffer'''

        # Check the start token is a0a2
        startToken = self.readUnsigned16BE()
        if startToken != 0xa0a2:
            raise ValueError('Unexpected start token 0x{:x} - expected 0x{:x}'.format(startToken, 0xa0a2))

        # Internal length is sometimes big endian, sometimes little endian!
        length = self.readUnsigned16BE()
        if length > 255:
            length = length >> 8
            
        # Main message content
        payload = self.readBytes(length)

        # Evaluate checksum
        checksum = self.readUnsigned16BE()
        if evaluateChecksum and checksum != sum(payload):
            raise ValueError('Checksum difference - {}'.format(checksum - sum(payload)))

        # Check the end token is b0b3
        endToken = self.readUnsigned16BE()
        if endToken != 0xb0b3:
            raise ValueError('Unexpected end token 0x{:x} - expected 0x{:x}'.format(endToken, 0xb0b3))

        return payload


    def consumeRawData(self):
        '''Consume raw SiRF data - convert to standard data structure'''

        # Convert HDOP from integer to decimal
        self.data['hdop'] = self.rawData['hdop'] / 5

        # Satellite count is already an integer
        self.data['sat'] = self.rawData['sv_count']

        # Copy satellite IDs
        self.data['sv_ids'] = self.rawData['sv_ids']

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['latitude'] / 10000000
        self.data['lon'] = self.rawData['longitude'] / 10000000

        # Convert elevation from cm to m
        self.data['ele'] = self.rawData['altitude_msl'] / 100

        # Convert SOG from cm/s to m/s
        self.data['sog'] = self.rawData['sog'] / 100

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['cog'] / 100

        # Convert rate of climb from cm/s to m/s
        self.data['roc'] = self.rawData['climb_rate'] / 100

        # Convert SDOP from cm/s to m/s - original SBN format was without SDOP
        if 'sdop' in self.rawData.dtype.names:
            self.data['sdop'] = self.rawData['sdop'] / 100

        # Convert VSDOP from cm/s to m/s - original SBN format was without VSDOP
        if 'vsdop' in self.rawData.dtype.names:
            self.data['vsdop'] = self.rawData['vsdop'] / 100

In [3]:
def getDevice(serial):
    '''Determine device type from serial number'''

    device = 'Unknown'
    
    if serial > 0:
        serialPrefix = serial // (10 ** floor(log10(serial) - 2))

        if serialPrefix == 100:
            device = 'GT-11'
        elif serialPrefix in [103, 113, 123, 133, 143, 832, 833, 932, 933]:
            device = 'GT-31'
        elif serialPrefix == 155:
            device = 'GW-52'
        elif serialPrefix == 168:
            device = 'GW-60'
        
    return device

## Unit Tests

In [4]:
class TestInit(unittest.TestCase):
    '''Class to test init'''

    def testFilename(self):
        '''Test the filename is as expected'''

        self.assertEqual(sirfReader.filename, 'test.gpx')


    def testHeader(self):
        '''Test the header is as expected'''

        self.assertEqual(sirfReader.header, {})


    def testData(self):
        '''Test the data is as expected'''

        self.assertEqual(sirfReader.data, {})


    def testNumRecords(self):
        '''Test the number of records is zero'''

        self.assertEqual(sirfReader.numRecords, 0)

In [5]:
class TestDevice(unittest.TestCase):
    '''Class to test device identification - GT-31 serials taken from Weymouth Speed Week 2010-2021'''

    def testGt11(self):
        '''Test GT-11'''

        self.assertEqual(getDevice(1001781), 'GT-11')
        self.assertEqual(getDevice(1003198), 'GT-11')


    def testGt31_103(self):
        '''Test GT-31 - 103 prefix'''

        self.assertEqual(getDevice(103200148), 'GT-31')
        self.assertEqual(getDevice(103201606), 'GT-31')


    def testGt31_113(self):
        '''Test GT-31 - 113 prefix'''

        self.assertEqual(getDevice(113200232), 'GT-31')
        self.assertEqual(getDevice(113300153), 'GT-31')


    def testGt31_123(self):
        '''Test GT-31 - 123 prefix'''

        self.assertEqual(getDevice(123200058), 'GT-31')
        self.assertEqual(getDevice(123300129), 'GT-31')


    def testGt31_133(self):
        '''Test GT-31 - 133 prefix'''

        self.assertEqual(getDevice(133200018), 'GT-31')
        self.assertEqual(getDevice(133201596), 'GT-31')


    def testGt31_143(self):
        '''Test GT-31 - 143 prefix'''

        self.assertEqual(getDevice(143200024), 'GT-31')
        self.assertEqual(getDevice(143200213), 'GT-31')


    def testGt31_832(self):
        '''Test GT-31 - 832 prefix'''

        self.assertEqual(getDevice(832000210), 'GT-31')
        self.assertEqual(getDevice(832004857), 'GT-31')


    def testGt31_833(self):
        '''Test GT-31 - 833 prefix'''

        self.assertEqual(getDevice(833000784), 'GT-31')
        self.assertEqual(getDevice(833002174), 'GT-31')


    def testGt31_932(self):
        '''Test GT-31 - 932 prefix'''

        self.assertEqual(getDevice(932000059), 'GT-31')
        self.assertEqual(getDevice(932001502), 'GT-31')


    def testGt31_933(self):
        '''Test GT-31 - 933 prefix'''

        self.assertEqual(getDevice(933000012), 'GT-31')


    def testGw52(self):
        '''Test GW-52'''

        self.assertEqual(getDevice(155800017), 'GW-52')


    def testGw60(self):
        '''Test GW-60'''

        self.assertEqual(getDevice(168601673), 'GW-60')
        self.assertEqual(getDevice(168605168), 'GW-60')

In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    sirfReader = SirfReader('test.gpx')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

................
----------------------------------------------------------------------
Ran 16 tests in 0.008s

OK
